In [77]:
import pandas as pd
import numpy as np
from data import * 
from inputs import *


In [78]:
def prep_for_calcs(df, yearly=False):
    # make a copy 
    df_df = df.copy()
    df_inputs = df.copy()

    if not yearly:
        # add columns for months and pre-fc 
        col_names = ["Pre-Financial Close", "Financial Close - July 2023"]
        # TODO add real months here!
        month_names = [f"Month {i}" for i in months]
        col_names = col_names + month_names
    else:
        col_names = ["Financial Close - July 2023", "Construction Year 1 - July 2024", "Commisioning and Construction Year 2 - July 2025"]
        year_names = [f"July {2025 + ix} - Year {ix}" for ix in year_nums]
        col_names = col_names + year_names


    # empty list to initialize 
    empty_list = len(df_df) * [0]


    # initialize df 
    for col in col_names:
        df_df[col] = empty_list

    # drop the value column 
    df_df.drop(columns=["Value"], inplace=True)

    return df_df, df_inputs

In [79]:
op_df, op_ref = prep_for_calcs(oper_phase_inputs, yearly=True)

In [80]:
# one time commisioning fees 
op_df.loc["Commisioning Fees", "Commisioning and Construction Year 2 - July 2025"] = op_ref.loc["Commisioning Fees","Value"].values

# quarterly payments to MaintCo 
op_df.iloc[op_df.index.get_level_values(1)=="Quarterly MaintCo Maintennance Fee", 3:] = op_ref.loc[("Operating Fees to External Entities", "Quarterly MaintCo Maintennance Fee"),"Value"]*4

# annual fishery payments 
op_df.iloc[op_df.index.get_level_values(1)=="Annual Fisheries Mitigation Permit", 3:] = op_ref.loc[("Operating Fees to External Entities", "Annual Fisheries Mitigation Permit"),"Value"]

# -- interconnection fee 
# decade one 
op_df.iloc[op_df.index.get_level_values(1)=="Monthly HIPU Interconection Fee, Decade 1", 3:3+10] = op_ref.loc[("Operating Fees to External Entities", "Monthly HIPU Interconection Fee, Decade 1"),"Value"]
# decade two 
op_df.iloc[op_df.index.get_level_values(1)=="Monthly HIPU Interconection Fee, Decade 1", 3+10:] = op_ref.loc[("Operating Fees to External Entities", "Monthly HIPU Interconection Fee, Post-Decade 1"),"Value"]
# rename
op_df.rename(index={'Monthly HIPU Interconection Fee, Decade 1': 'Monthly HIPU Interconection Fee'}, inplace=True)
# drop the other entry 
op_df.drop("Monthly HIPU Interconection Fee, Post-Decade 1", level=1, inplace=True)


In [87]:
op_df

Financial Close - July 2023  \
Commisioning Fees                   ABSCO Interconnection Fee                                     0   
                                    ABSCO Network Upgrades                                        0   
                                    HIPU Interconnection Fee                                      0   
Operating Fees to External Entities Quarterly MaintCo Maintennance Fee                            0   
                                    Annual Fisheries Mitigation Permit                            0   
                                    Monthly HIPU Interconection Fee                               0   

                                                                        Construction Year 1 - July 2024  \
Commisioning Fees                   ABSCO Interconnection Fee                                         0   
                                    ABSCO Network Upgrades                                            0   
                                    HIPU Interconnection Fee                                          0   
Operating Fees to External Entities Quarterly MaintCo Maintennance Fee                                0   
                                    Annual Fisheries Mitigation Permit                                0   
                                    Monthly HIPU Interconection Fee                                   0   

                                                                        Commisioning and Construction Year 2 - July 2025  \
Commisioning Fees                   ABSCO Interconnection Fee                                                    2500000   
                                    ABSCO Network Upgrades                                                       5200000   
                                    HIPU Interconnection Fee                                                     2000000   
Operating Fees to External Entities Quarterly MaintCo Maintennance Fee                                                 0   
                                    Annual Fisheries Mitigation Permit                                                 0   
                                    Monthly HIPU Interconection Fee                                                    0   

                                                                        July 2026 - Year 1  \
Commisioning Fees                   ABSCO Interconnection Fee                            0   
                                    ABSCO Network Upgrades                               0   
                                    HIPU Interconnection Fee                             0   
Operating Fees to External Entities Quarterly MaintCo Maintennance Fee             2760000   
                                    Annual Fisheries Mitigation Permit              130000   
                                    Monthly HIPU Interconection Fee                 110000   

                                                                        July 2027 - Year 2  \
Commisioning Fees                   ABSCO Interconnection Fee                            0   
                                    ABSCO Network Upgrades                               0   
                                    HIPU Interconnection Fee                             0   
Operating Fees to External Entities Quarterly MaintCo Maintennance Fee             2760000   
                                    Annual Fisheries Mitigation Permit              130000   
                                    Monthly HIPU Interconection Fee                 110000   

                                                                        July 2028 - Year 3  \
Commisioning Fees                   ABSCO Interconnection Fee                            0   
                                    ABSCO Network Upgrades                               0   
                                    HIPU Interconnection Fee                             0   
Operating Fees to External Entities Q

# Construction Phase

In [88]:
const_df, const_inputs = prep_for_calcs(const_phase_inputs)

In [89]:
# pre- financial close
const_df.loc["Pre-Financial Close Expenses", "Pre-Financial Close"] = const_inputs.loc["Pre-Financial Close Expenses"]["Value"].values

# at financial close 
const_df.loc["Financial Close Expenses", "Financial Close - July 2023"] = const_inputs.loc["Financial Close Expenses"]["Value"].values

In [90]:
# monthly split expenses 
vals = const_inputs.loc["Construction Period Even Split Monthly Expenses"]["Value"].values

a = const_df.loc["Construction Period Even Split Monthly Expenses"].apply(lambda x: vals/12)

for i in const_df.loc["Construction Period Even Split Monthly Expenses"].index:
    const_df.loc[("Construction Period Even Split Monthly Expenses", i)] = a.loc[i]

const_df.loc["Construction Period Even Split Monthly Expenses", ["Pre-Financial Close", "Financial Close - July 2023"]] = 0


In [91]:
# construction expenses 
for ix, col in enumerate(const_df.columns):
    ix1 = "Construction Period Custom Schedule Monthly Expenses"
    ix2 = "EPC Cost"
    val = const_inputs.loc[(ix1, ix2), "Value"]

    offset = 1
    if ix <= offset:
        const_df.loc[(ix1, ix2), col]  = 0
    else:
        const_df.loc[(ix1, ix2), col]  = epc_sched[ix -1] * val

    # ref = 2



In [92]:
const_df.loc[("Total", ""), :] = const_df.sum(axis=0)
const_df

Pre-Financial Close  \
Pre-Financial Close Expenses                       Wetland Construction Permit                   3000000.0   
                                                   State Environmental Monitoring Fee             800000.0   
Financial Close Expenses                           Land                                                0.0   
                                                   Development Expenses                                0.0   
                                                   Development Fees                                    0.0   
                                                   Closing Costs                                       0.0   
                                                   Contingency Fees                                    0.0   
Construction Period Even Split Monthly Expenses    Interest and Fees                                   0.0   
                                                   Insurance and Fees                                  0.0   
                                                   Management and Oversight                            0.0   
Construction Period Custom Schedule Monthly Exp... EPC Cost                                            0.0   
Total                                                                                            3800000.0   

                                                                                       Financial Close - July 2023  \
Pre-Financial Close Expenses                       Wetland Construction Permit                                 0.0   
                                                   State Environmental Monitoring Fee                          0.0   
Financial Close Expenses                           Land                                                 12500000.0   
                                                   Development Expenses                                 28500000.0   
                                                   Development Fees                                     14700000.0   
                                                   Closing Costs                                        14392000.0   
                                                   Contingency Fees                                     30754000.0   
Construction Period Even Split Monthly Expenses    Interest and Fees                                           0.0   
                                                   Insurance and Fees                                          0.0   
                                                   Management and Oversight                                    0.0   
Construction Period Custom Schedule Monthly Exp... EPC Cost                                                    0.0   
Total                                                                                                  100846000.0   

                                                                                            Month 1  \
Pre-Financial Close Expenses                       Wetland Construction Permit         0.000000e+00   
                                                   State Environmental Monitoring Fee  0.000000e+00   
Financial Close Expenses                           Land                                0.000000e+00   
                                                   Development Expenses                0.000000e+00   
                                                   Development Fees                    0.000000e+00   
                                                   Closing Costs                       0.000000e+00   
                                                   Contingency Fees                    0.000000e+00   
Construction Period Even Split Monthly Expenses    Interest and Fees                   4.737750e+06   
                                                   Insurance and Fees                  2.083333e+06   
                                                   Management and Oversight            6.79

In [93]:
# 